<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Define-dataframes-and-function" data-toc-modified-id="Define-dataframes-and-function-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Define dataframes and function</a></span><ul class="toc-item"><li><span><a href="#Read-in-data" data-toc-modified-id="Read-in-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Read in data</a></span></li><li><span><a href="#Combine-pieces-from-parallel-runs" data-toc-modified-id="Combine-pieces-from-parallel-runs-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Combine pieces from parallel runs</a></span></li><li><span><a href="#print_row_detail-function" data-toc-modified-id="print_row_detail-function-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>print_row_detail function</a></span></li></ul></li><li><span><a href="#Test-and-refine-regex-patterns-for-flagging-likely-relevant-documents" data-toc-modified-id="Test-and-refine-regex-patterns-for-flagging-likely-relevant-documents-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test and refine regex patterns for flagging likely relevant documents</a></span><ul class="toc-item"><li><span><a href="#Working-list-for-flagging-positive-matches" data-toc-modified-id="Working-list-for-flagging-positive-matches-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Working list for flagging positive matches</a></span></li><li><span><a href="#Test-cases-for-regex" data-toc-modified-id="Test-cases-for-regex-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Test cases for regex</a></span></li><li><span><a href="#Assign-initial-positive-and-negative-labels-using-regex" data-toc-modified-id="Assign-initial-positive-and-negative-labels-using-regex-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Assign initial positive and negative labels using regex</a></span><ul class="toc-item"><li><span><a href="#function:-label_with_regex" data-toc-modified-id="function:-label_with_regex-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>function: label_with_regex</a></span></li></ul></li><li><span><a href="#Identify-and-remove-bad-training-examples" data-toc-modified-id="Identify-and-remove-bad-training-examples-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Identify and remove bad training examples</a></span></li><li><span><a href="#Check-for-false-positives" data-toc-modified-id="Check-for-false-positives-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Check for false positives</a></span></li></ul></li><li><span><a href="#Write-current-work-to-csv" data-toc-modified-id="Write-current-work-to-csv-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Write current work to csv</a></span></li></ul></div>

# Label paragraphs

This notebook labels paragraphs as "relevant" (positive hits) or not. 
Relevant = contains the employee count for the entire company for the most recent year. 

To do:
- Modify print_row_detail function so that it can return the row index
    - Needed in order to create a list of false positives or negatives to correct

## Define dataframes and function

### Read in data

In [1]:
import pandas as pd

train_accession_ids = pd.read_csv('../data/train_accession_ids.csv', names=['acc_id'])['acc_id'].tolist()
val_accession_ids = pd.read_csv('../data/val_accession_ids.csv', names=['acc_id'])['acc_id'].tolist()

# Read in labeled data, split into seprate training and validation dataframes
labeled_df = pd.read_excel('../data/train_val_employee_count_paragraphs.xlsx')
labeled_train_df = labeled_df.loc[labeled_df.split == 'train', :].copy()
labeled_val_df = labeled_df.loc[labeled_df.split == 'val', :].copy()

We also need the paragraphs that were parsed from the html files. 

In [2]:
# After reading back in, the html needs to be parsed again
from bs4 import BeautifulSoup as bs

tbl_html_df = pd.read_csv('data/tbl_html_df3.csv')
tbl_html_df['tbl_html'] = tbl_html_df.tbl_html.apply(lambda x: bs(x, 'lxml'))

# Set the data types for the columns
p_dtypes = {'acc_id' : 'category', 
'len' : 'int', 
'emp_header' : 'bool', 
'first_emp_head_block' : 'bool', 
'split' : 'category', 
'label' : 'uint8'}

# Read in the paragraphs, then separate the training and validation sets
paragraph_input_df = pd.read_csv('data/paragraph_input_df3.csv', dtype=p_dtypes, index_col=0)
train_df = paragraph_input_df[paragraph_input_df.split == 'train']
val_df = paragraph_input_df[paragraph_input_df.split == 'val']

### Combine pieces from parallel runs

In [56]:
"""
paragraph_input_df_1 = pd.read_csv('data/paragraph_input_df_1.csv', dtype=p_dtypes, index_col=0)
paragraph_input_df_2 = pd.read_csv('data/paragraph_input_df_2.csv', dtype=p_dtypes, index_col=0)
paragraph_input_df_3 = pd.read_csv('data/paragraph_input_df_3.csv', dtype=p_dtypes, index_col=0)
paragraph_input_df_4 = pd.read_csv('data/paragraph_input_df_4.csv', dtype=p_dtypes, index_col=0)

print(paragraph_input_df.shape)
print(paragraph_input_df_1.shape)
print(paragraph_input_df_2.shape)
print(paragraph_input_df_3.shape)
print(paragraph_input_df_4.shape)
"""

(9217, 9)
(3775, 10)
(4132, 10)
(4442, 10)
(3996, 10)


In [63]:
"""
tbl_html_df_1 = pd.read_csv('data/tbl_html_df_1.csv', dtype=p_dtypes, index_col=0)
tbl_html_df_2 = pd.read_csv('data/tbl_html_df_2.csv', dtype=p_dtypes, index_col=0)
tbl_html_df_3 = pd.read_csv('data/tbl_html_df_3.csv', dtype=p_dtypes, index_col=0)
tbl_html_df_4 = pd.read_csv('data/tbl_html_df_4.csv', dtype=p_dtypes, index_col=0)

print(tbl_html_df.shape)
print(tbl_html_df_1.shape)
print(tbl_html_df_2.shape)
print(tbl_html_df_3.shape)
print(tbl_html_df_4.shape)
"""

(208, 4)
(57, 4)
(57, 4)
(26, 4)
(81, 4)


In [64]:
"""
tbl_html_df_1['idx'] = tbl_html_df_1.index.values
tbl_html_df_2['idx'] = tbl_html_df_2.index.values + tbl_html_df_1.shape[0] 
tbl_html_df_3['idx'] = tbl_html_df_3.index.values + tbl_html_df_2.shape[0] + tbl_html_df_1.shape[0]
tbl_html_df_4['idx'] = tbl_html_df_4.index.values + tbl_html_df_3.shape[0] + tbl_html_df_2.shape[0] + tbl_html_df_1.shape[0]
print(tbl_html_df_1.head(1)['idx'])
print(tbl_html_df_1.tail(1)['idx'])
print(tbl_html_df_2.head(1)['idx'])
print(tbl_html_df_2.tail(1)['idx'])
print(tbl_html_df_3.head(1)['idx'])
print(tbl_html_df_3.tail(1)['idx'])
print(tbl_html_df_4.head(1)['idx'])
print(tbl_html_df_4.tail(1)['idx'])
"""

0    0
Name: idx, dtype: int64
56    56
Name: idx, dtype: int64
0    57
Name: idx, dtype: int64
56    113
Name: idx, dtype: int64
0    114
Name: idx, dtype: int64
25    139
Name: idx, dtype: int64
0    140
Name: idx, dtype: int64
80    220
Name: idx, dtype: int64


In [57]:
"""
paragraph_input_df_2['idx'] = paragraph_input_df_2.index.values + paragraph_input_df_1.shape[0] 
paragraph_input_df_3['idx'] = paragraph_input_df_3.index.values + paragraph_input_df_2.shape[0] + paragraph_input_df_1.shape[0]
paragraph_input_df_4['idx'] = paragraph_input_df_4.index.values + paragraph_input_df_3.shape[0] + paragraph_input_df_2.shape[0] + paragraph_input_df_1.shape[0]
print(paragraph_input_df_1.head(1)['idx'])
print(paragraph_input_df_1.tail(1)['idx'])
print(paragraph_input_df_2.head(1)['idx'])
print(paragraph_input_df_2.tail(1)['idx'])
print(paragraph_input_df_3.head(1)['idx'])
print(paragraph_input_df_3.tail(1)['idx'])
print(paragraph_input_df_4.head(1)['idx'])
print(paragraph_input_df_4.tail(1)['idx'])
"""

0    0
Name: idx, dtype: int64
3774    3774
Name: idx, dtype: int64
0    3775
Name: idx, dtype: int64
4131    7906
Name: idx, dtype: int64
0    7907
Name: idx, dtype: int64
4441    12348
Name: idx, dtype: int64
0    12349
Name: idx, dtype: int64
3995    16344
Name: idx, dtype: int64


In [58]:
#tbl_html_df3 = pd.concat([tbl_html_df_1, tbl_html_df_2, tbl_html_df_3, tbl_html_df_4]).set_index('idx', drop=True)
#tbl_html_df3.to_csv('data/tbl_html_df3.csv')
#paragraph_input_df3 = pd.concat([paragraph_input_df_1, paragraph_input_df_2, paragraph_input_df_3, paragraph_input_df_4]).set_index('idx', drop=True)
#paragraph_input_df3.to_csv('data/paragraph_input_df3.csv')

### print_row_detail function

In [3]:
def print_row_detail(df=train_df, nrow=10, header_list = ['acc_id'],
                    detail_list = [ 'len', 'first_emp_head_block', 'split' ,'para_text'],
                    sortby=['acc_id', 'first_emp_head_block', 'len'], ascending=False):
    df_sorted = df.sort_values(sortby, ascending=ascending).reset_index()
    nrow = min(len(df_sorted), nrow)
    for i in range(0, nrow):
        for h in header_list:
            print('-'*35  + ' ' +  str(df_sorted[h][i]) + ' ' + '-'*35)
        for d in detail_list:
            print(d + '  :' + str(df_sorted[d][i]))
            print('')

print_row_detail()

----------------------------------- 0001703056-18-000008 -----------------------------------
len  :241

first_emp_head_block  :True

split  :train

para_text  :As of December 31, 2017, we employed approximately 18,000 people. Approximately 10% of our employees are covered by collective bargaining agreements. We believe that our relations with our employees and labor unions have generally been good.

----------------------------------- 0001703056-18-000008 -----------------------------------
len  :1249

first_emp_head_block  :False

split  :train

para_text  :James D. DeVries is our President. From May 2, 2016 to September 2017, Mr. DeVries served as The ADT Corporation's Executive Vice President and Chief Operating Officer. From December 2014 to May 2016, Mr. DeVries served as Executive Vice President of Brand Operations at Allstate Insurance Company, the second largest personal lines insurer in the United States. During his tenure at Allstate, Mr. DeVries led the operations organizati

## Test and refine regex patterns for flagging likely relevant documents

### Working list for flagging positive matches

In [4]:
import re 
# Initial components
employee_terms = "(associates|employees|full[ -]time[ -]equivalent(s)?|staff|team members|workers)"
person_terms = "(individuals|people|persons)" # These need additional cues
workforce_terms = "(((employee|employment|head|personnel|staff|worker|workforce) (count(s)|level(s)|total(s))+)|(head-count|headcount|workforce))"
employee_type_terms = "(full time|full-time|permanent|part time|part-time|regular|seasonal|temporary|total)"

numeral_pat = "(([0-9]{1,3},)*[0-9]{1,3}([.][0-9])?)" # Include numerals, requiring comma separation when appropriate, and allowing for decimals. 
rel_qualifiers = "(a total of|approximately|in aggregate|in total|(an|the) equivalent of|total)"

space_pat = "( |\n)" # In html, a space is often missing if the paragraph continues on the next line

magnitude_words = "(hundred|thousand|million|billion)" 
num_words = "(one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen)"

# Composites to allow numerals and/or number words with relative modifiers
num_pat = "".join(["((",numeral_pat, "|", num_words, ")(", space_pat, magnitude_words, ")?", ")"])
num_emps_pat = "".join([num_pat, space_pat, "(", rel_qualifiers, space_pat, ")*", 
                        "(", employee_type_terms, space_pat, ")*"])

# Actual patterns to be used, named by the format they're meant to capture
number_employees_pat = "".join([num_emps_pat, employee_terms])
employed_num_pat = "".join(["employ((ed|s)?)?", space_pat, "(", rel_qualifiers, space_pat, ")*", 
                        num_emps_pat])
employed_words_num_pat = "".join(["employ((ed|s)?)?", "[^.]+ *", numeral_pat])
workforce_words_num_pat = "".join([workforce_terms, "[^.]+ *", numeral_pat])

# Pattern list, including some raw regular expressions
flag_pat_list = [number_employees_pat, 
                employed_num_pat, 
                employed_words_num_pat, 
                workforce_words_num_pat, 
               r"((number of|head[ -]?count|total|workforce) )[^.]+ (employees|team members) [^.]+ ([0-9]{1,3},)*[0-9]{1,3}[^\d]", 
               r"(employees)(of the company )?((is|was|approximately) )*([0-9]{1,3},)*[0-9]{1,3}"
]
flag_pats = [re.compile(x, re.I) for x in flag_pat_list]

In [5]:
# Original list, before modularized regex
flag_pat_list_original = [
r"employ((ed|s)?) ((approximately|the equivalent of) )*([0-9]{1,3},)*[0-9]{1,3} ((permanent|full|time|part|time|temporary|total|and|equivalent)[ -])*(employees|individuals|people|team members|members|persons|associates)",
r"([0-9]{1,3},)*[0-9]{1,3} [^.]+ ((employ\w*|head([ -])?count|member(s)?|person(s)?|people|staff|team|workforce))",
r"((employ(ed)?|employee count|head([ -])?count|member(s)?|person(s)?|people|staff|team|workforce) )+[^.]+ *([0-9]{1,3},)*[0-9]{1,3}",
r"((number of|head[ -]?count|total|workforce) )[^.]+ (employees|team members) [^.]+ ([0-9]{1,3},)*[0-9]{1,3}[^\d]",
r"([0-9]{1,3},)*[0-9]{1,3} ((permanent|full|time|part|time|temporary|total|and|equivalent)[ -])*(employees|team members)",
r"(employees)(of the company )?((is|was|approximately) )*([0-9]{1,3},)*[0-9]{1,3}"]


#r"((permanent|full|part|time|full-time|part-time|temporary) )+(employees|team members|associates)",
#r"employed( approximately)?$", 
#r"Total workforce",

#r"^((permanent|full|part|time|full-time|part-time|temporary|total) )*(employees|team members|associates)"
                 
flag_pats_original = [re.compile(x, re.I) for x in flag_pat_list_original]

### Test cases for regex

In [6]:

def check_regex_match(pattern, text_list):
    for idx, s in enumerate(text_list):
        mo = re.search(pattern, s)
        if mo:
            ms = mo.span()[1]
            print("------    " + str(idx) + "   Matched!    -----")
            print('str length  :' + str(len(s)) + '    match span  :' + str(ms))
            print(s[:ms])
            print(s[ms:])
            print(re.search(pattern, s))
        else:
            print("------    " + str(idx) + "  NO MATCH    -----")
            print(s)

test_list = ["We are a small company with approximately 61 employees.",
"The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015. "
"Total workforce level at December 31, 2016 was approximately 150,500",           
"Currently, the Company and its subsidiaries have an aggregate of 35 employees.",
"we employ only 31 employees", 
"We currently have 21 employees",
"We currently employ 26 full-time employees",
"Including our full and part-time personnel, we estimate that we have the equivalent of 12 full time employees."
"As a REIT, we employ only 31 employees and have a cost-effective management structure."
]

check_regex_match(re.compile(employed_words_num_pat, re.I), test_list)

------    0  NO MATCH    -----
We are a small company with approximately 61 employees.
------    1   Matched!    -----
str length  :196    match span  :126
The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015
. Total workforce level at December 31, 2016 was approximately 150,500
<_sre.SRE_Match object; span=(24, 126), match='employees of the Company was approximately 31,800>
------    2  NO MATCH    -----
Currently, the Company and its subsidiaries have an aggregate of 35 employees.
------    3   Matched!    -----
str length  :27    match span  :17
we employ only 31
 employees
<_sre.SRE_Match object; span=(3, 17), match='employ only 31'>
------    4  NO MATCH    -----
We currently have 21 employees
------    5   Matched!    -----
str length  :42    match span  :22
We currently employ 26
 full-time employees
<_sre.SRE_Match object; span=(13, 22), match='employ 26'>
------    6   Matched!    -----
str length  :196 

### Assign initial positive and negative labels using regex

#### function: label_with_regex

In [7]:
def label_with_regex(df, regex_list, text_col = 'para_text', label_col = 'label', 
                    reset_labels_first = True):
    """Label df in place if text matches any regex in regex_list"""
    df_para_list = df[text_col].tolist()
    pos_ids = set()
    for i, para in enumerate(df_para_list):
    ## "any() does not work for this purpose - it takes some explosive logic path"
    #if not any(reg.search(para) for reg in nonum_regs) and k2 not in para_dict[k]['df'].keys():
        for reg in regex_list:
            if reg.search(para):
                pos_ids.add(df.index[i])
                break
    if reset_labels_first:
        df.loc[:, label_col] = 0
    df.loc[df.index.isin(pos_ids), label_col] = 1

# Apply labels to training and validation sets
label_with_regex(train_df, flag_pats)
label_with_regex(val_df, flag_pats)

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Identify and remove bad training examples

Short "paragraphs" are usually headings (or parts of tables)\*, so they can be eliminated. Otherwise, they will add "noise" when using machine learning. If they are left in, then a machine learing algorithm will "learn" what makes relevant paragraphs (i.e., paragraphs that contain employee count information) different from section headings. 

A relevant paragraph could conceivably have 16 characters, as in this fictitious example:  
> Headcount was 5.

The shortest useful paragraph probably has 19, as in this ictitious example:  
> We had 9 employees. 

As shown below, there are no such examples in the training set. Thus, leaving the short negative (i.e., irrelevant) examples in the dataset 

\* The calculated length sometimes includes newline characters (i.e., invisible characters), primarily from tables. Tables are exlcuded from this project, as table parsing is a separate (and much more solved) task. 

In [8]:
import numpy as np 
train_df[np.logical_and(train_df.label ==0, # None of the paragraphs with positive labels are this short
                        train_df.len <= 19)][train_df.len >11].para_text.value_counts().to_frame()

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,para_text
D. Employees,19
Total employees,8
Workforce Solutions,7
Total Employees,6
D. Employees,6
Capital employed,5
Workforce reduction,5
Team Members,4
Workforce Reduction,4
D. EMPLOYEES,3


In [9]:
# Remove short paragraphs
train_df = train_df.loc[train_df.len > 19, :]
val_df = val_df.loc[val_df.len > 19, :]

Since we want paragraphs with employee count data, paragraphs with few or no numerals should be examined for exclusion.  all paragraphs without numbers can be removed from the negative examples. 

In [10]:
nonum_pats = [r"^([^\d]|[$][\d]+)[^\d]+$", # No numerals, except for a dollar amount at the begining
    r"^[^\d]{0,2}\d{1,2}[^\d]*$", 
    r"^(([^\d]{0,2}\d{1,2})|[^\d]|[$][\d]+)[^\d]+$", # Combines the first two patterns
    r"^[^\d]* \d{1,2}[^\d]*$", 
    r"^([^\d]+(\d{1,2}[^\d]{2,}){1,5}\d{1,3}[^\d]*)$",
    r"^((\d{1,3}[^\d]{2,}){1,4}\d{1,2}[^\d]*)$"]
year_and_num = re.compile(r"20[0-2][0-9].*[0-9]{1,3}.*|[0-9]{1,3}.*20[0-2][0-9]", re.I)
nonum_regs = [re.compile(x) for x in nonum_pats]

print("Matches among paragraphs labeled as negative hits:")
for y in nonum_regs:
    print(train_df[(train_df.label == 0) & (train_df.para_text.str.match(y) == True)].shape[0])

print("Matches among paragraphs labeled as positve hits:")
for y in nonum_regs:
    print(train_df[(train_df.label == 1) & (train_df.para_text.str.match(y) == True)].shape[0])

Matches among paragraphs labeled as negative hits:
2806
25
2829
114
80
2
Matches among paragraphs labeled as positve hits:
71
7
78
79
94
3


None of the paragraphs without numerals (outside of dollar amounts or section numbers) are relevant.

In [11]:
check_pat = nonum_regs[2]
check_regex_match(check_pat, [x for x in train_df.loc[train_df.label == 1, :].para_text.tolist() if re.search(check_pat, x)])

------    0   Matched!    -----
str length  :90    match span  :90
2    Shares held by two employees of the Company to satisfy the requirements of local law.

<_sre.SRE_Match object; span=(0, 90), match='2    Shares held by two employees of the Company >
------    1   Matched!    -----
str length  :1074    match span  :1074
In determining the fair market value of securities held in the Trust Fund which are listed on a registered stock exchange, the Administrator shall direct the Trustee to value the same at the prices they were last traded on such exchange preceding the close of business on the Valuation Date. If such securities were not traded on the Valuation Date, or if the exchange on which they are traded was not open for business on the Valuation Date, then the securities shall be valued at the prices at which they were last traded prior to the Valuation Date. Any unlisted security held in the Trust Fund shall be valued at its bid price next preceding the close of business on the

In [12]:
# Pattern to identify paragraphs that either have no numerals or 
# only have numerals as aprt of dollar amounts
nonum_pat = re.compile(r"^(([^\d]{0,2}\d{1,2})|[^\d]|[$][\d]+)[^\d]+$", re.I)

# Index of rows to drop 
nonum_idx = train_df[train_df.para_text.str.match(nonum_pat) == True].index
nonum_idx_val = val_df[val_df.para_text.str.match(nonum_pat) == True].index
# Remove rows 
train_df = train_df.drop(nonum_idx)
val_df = val_df.drop(nonum_idx_val)

Training set:

In [13]:
print('Total paragraphs: ', train_df.shape[0])
print('Positive paragraphs: ',  train_df[train_df.label == 1].shape[0])
print('Total keys: ', len(train_df.acc_id.unique()))
print('Keys with positive labels: ', len(train_df[train_df.label ==1].acc_id.unique()))
print('Negative paragraphs: ',  train_df[train_df.label == 0].shape[0])
print('Keys with negative labels: ', len(train_df[train_df.label ==0].acc_id.unique()))

Total paragraphs:  7700
Positive paragraphs:  5331
Total keys:  1644
Keys with positive labels:  1618
Negative paragraphs:  2369
Keys with negative labels:  905


Validation set:

In [14]:
print('Total paragraphs: ', val_df.shape[0])
print('Positive paragraphs: ',  val_df[val_df.label == 1].shape[0])
print('Total keys: ', len(val_df.acc_id.unique()))
print('Keys with positive labels: ', len(val_df[val_df.label ==1].acc_id.unique()))
print('Negative paragraphs: ',  val_df[val_df.label == 0].shape[0])
print('Keys with negative labels: ', len(val_df[val_df.label ==0].acc_id.unique()))

Total paragraphs:  2657
Positive paragraphs:  1856
Total keys:  552
Keys with positive labels:  543
Negative paragraphs:  801
Keys with negative labels:  323


### Check for false positives

In [15]:
row_filter = np.logical_and(train_df.label == 1, 
    train_df.para_text.apply(lambda x: bool(re.search(nonum_pats[3], x)))
                           )

print_row_detail(df=train_df[row_filter], nrow=50, header_list = ['acc_id'],
                    detail_list = [ 'len', 'first_emp_head_block','para_text'],
                    sortby=['first_emp_head_block','acc_id',  'len'], ascending=False)

----------------------------------- 0001527709-17-000019 -----------------------------------
len  :544

first_emp_head_block  :True

para_text  :The officers of our general partner manage our operations and activities. Neither we, our subsidiaries, nor our general partner have employees. Our general partner has entered into a services agreement with Mid-Con Energy Operating pursuant to which Mid-Con Energy Operating will perform services for us, including the operation of our properties. Mid-Con Energy Operating has over 70 employees performing services for our operations and activities. We believe that Mid-Con Energy Operating has a satisfactory relationship with these employees.

----------------------------------- 0001474098-17-000010 -----------------------------------
len  :255

first_emp_head_block  :True

para_text  :We currently employ 26 full-time employees. None of our employees is a member of a union; however, some employees of the hotel managers at several of our hotels are

## Write current work to csv

This allows code cleanup in this notebook while tuning classifiers in another notebook

In [16]:
train_df.to_csv('data/classifier_input_train3.csv')
val_df.to_csv('data/classifier_input_val3.csv')